In [ ]:
## Code for building detection and cropping 
## Article: Developing Building Exposure Models Using Computer Vision and Deep Learning
#
# Authors: Sukh Sagar Shukla, Amit Bhatiya, Dhanya J, Saman Ghaffarian, Roberto Gentile
#
# Description:
# This script detects and saves the images using the Faster-RCNN object detection model for panoramic images  
# Please refer to the article for further details.


import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

# Function to crop and save the image
def crop_and_save(original_image, box, save_path, expand_factor=0.1, min_dim=200):
    img_height, img_width, _ = original_image.shape
    ymin, xmin, ymax, xmax = box

    # Calculate expansion in both height and width
    box_width = xmax - xmin
    box_height = ymax - ymin
    expand_w = box_width * expand_factor
    expand_h = box_height * expand_factor

    # Expand the bounding box
    xmin_expanded = max(0, xmin - expand_w)
    ymin_expanded = max(0, ymin - expand_h)
    xmax_expanded = min(1, xmax + expand_w)
    ymax_expanded = min(1, ymax + expand_h)

    # Convert normalized coordinates back to image coordinates
    xmin_pixel = int(xmin_expanded * img_width)
    ymin_pixel = int(ymin_expanded * img_height)
    xmax_pixel = int(xmax_expanded * img_width)
    ymax_pixel = int(ymax_expanded * img_height)

    # Crop the image using the expanded bounding box
    cropped_image = original_image[ymin_pixel:ymax_pixel, xmin_pixel:xmax_pixel]
    
    cropped_height, cropped_width, _ = cropped_image.shape

    # Check if the cropped image meets the minimum dimension requirement
    if cropped_width < min_dim or cropped_height < min_dim:
        return

    # Convert back to uint8 and scale to 0-255 range
    cropped_image = (cropped_image * 255).astype(np.uint8)

    # Save the cropped image
    try:
        cropped_image_pil = Image.fromarray(cropped_image)
        cropped_image_pil.save(save_path, format="JPEG", quality=180)
    except Exception as e:
        print(f"Error saving image: {str(e)}")

# Function to calculate intersection over union (IoU)
def calculate_iou(box1, box2):
    y1_1, x1_1, y2_1, x2_1 = box1
    y1_2, x1_2, y2_2, x2_2 = box2

    intersection_y1 = max(y1_1, y1_2)
    intersection_x1 = max(x1_1, x1_2)
    intersection_y2 = min(y2_1, y2_2)
    intersection_x2 = min(x2_1, x2_2)

    intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)
    
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    union_area = box1_area + box2_area - intersection_area
    
    iou = intersection_area / union_area if union_area > 0 else 0
    return iou

# Define the target classes
TARGET_CLASSES = ['House', 'Building', 'Skyscraper', 'Tower']

# Load the TensorFlow Hub model
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

# Define the directories for input and output images
image_dir_path = 'path/to/your/panorma/images' 
image_dir_path_2 = 'path/to/save/the/detected/building/images'
os.makedirs(image_dir_path_2, exist_ok=True)

# Get the list of image files in the input directory
image_files = [f for f in os.listdir(image_dir_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Process each image
for image_file in image_files:
    image_path = os.path.join(image_dir_path, image_file)

    # Load and preprocess the image
    print(f"Loading and preprocessing image: {image_path}")
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, (3600, 3600))
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    image = tf.cast(image, tf.float32) / 255.0  # Normalize the image

    # Run object detection
    output_dict = detector(image)

    # Extract relevant outputs
    boxes = output_dict['detection_boxes'].numpy()
    scores = output_dict['detection_scores'].numpy()
    classes = output_dict['detection_class_entities'].numpy()

    # Set a threshold and select detected boxes
    threshold = 0.3
    detected_indices = np.where(scores >= threshold)[0]

    original_image = tf.squeeze(image).numpy()

    final_detections = []
    iou_threshold = 0.5
    original_image_name = os.path.splitext(image_file)[0]

    # Process detections
    for i, detection_index in enumerate(detected_indices):
        class_name = classes[detection_index].decode('utf-8').capitalize()
        
        if class_name in TARGET_CLASSES:
            current_box = boxes[detection_index]
            current_score = scores[detection_index]
            
            is_duplicate = False
            for existing_detection in final_detections:
                if calculate_iou(current_box, existing_detection['box']) > iou_threshold:
                    if current_score > existing_detection['score']:
                        final_detections.remove(existing_detection)
                        final_detections.append({
                            'class': class_name,
                            'box': current_box,
                            'score': current_score,
                            'index': i
                        })
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                final_detections.append({
                    'class': class_name,
                    'box': current_box,
                    'score': current_score,
                    'index': i
                })

    # Save final cropped detections
    for detection_index, detection in enumerate(final_detections):
        class_name = detection['class']
        box = detection['box']
        score = detection['score']

        cropped_image_save_path = os.path.join(image_dir_path_2, f'{original_image_name}-{detection_index+1}.jpg')

        # Crop and save the bounding box image
        crop_and_save(original_image, box, cropped_image_save_path, expand_factor=0.1, min_dim=200)

print("Processing complete for all images.")